# Metin Sınıflandırma Görevi

Daha önce belirttiğimiz gibi, **AG_NEWS** veri setine dayalı basit bir metin sınıflandırma görevi üzerinde duracağız. Bu görev, haber başlıklarını Dünya, Spor, İş ve Bilim/Teknoloji olmak üzere 4 kategoriden birine sınıflandırmaktır.

## Veri Seti

Bu veri seti [`torchtext`](https://github.com/pytorch/text) modülüne dahil edilmiştir, bu yüzden kolayca erişebiliriz.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Burada, `train_dataset` ve `test_dataset` sırasıyla etiket (sınıf numarası) ve metin çiftlerini döndüren koleksiyonlar içerir, örneğin:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Öyleyse, veri setimizden ilk 10 yeni başlığı yazdıralım:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Veri kümeleri yineleyici olduğundan, verileri birden fazla kez kullanmak istiyorsak listeye dönüştürmemiz gerekir:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenizasyon

Şimdi metni tensörler olarak temsil edilebilecek **sayılara** dönüştürmemiz gerekiyor. Eğer kelime düzeyinde bir temsil istiyorsak, iki şey yapmamız gerekiyor:
* Metni **tokenizer** kullanarak **tokenlere** ayırmak  
* Bu tokenlerin bir **vokabülerini** oluşturmak.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Kelime dağarcığını kullanarak, tokenleştirilmiş dizimizi kolayca bir sayı kümesine kodlayabiliriz:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Kelime Torbası (Bag of Words) metin temsili

Kelimeler anlamı temsil ettiği için, bazen bir metnin anlamını cümledeki sıralarına bakmaksızın sadece bireysel kelimelere bakarak anlayabiliriz. Örneğin, haberleri sınıflandırırken, *hava durumu*, *kar* gibi kelimeler muhtemelen *hava durumu tahmini*ni işaret ederken, *hisse senetleri*, *dolar* gibi kelimeler *finans haberleri*ne işaret edebilir.

**Kelime Torbası** (BoW) vektör temsili, en yaygın kullanılan geleneksel vektör temsilidir. Her kelime bir vektör indeksine bağlanır ve vektör elemanı, bir belgedeki bir kelimenin kaç kez geçtiğini içerir.

![Kelime torbası vektör temsilinin bellekte nasıl temsil edildiğini gösteren bir görsel.](../../../../../translated_images/tr/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Not**: BoW'yu, metindeki bireysel kelimeler için tekil olarak bir-hot kodlanmış vektörlerin toplamı olarak da düşünebilirsiniz.

Aşağıda, Scikit Learn Python kütüphanesi kullanılarak bir kelime torbası temsili oluşturmanın bir örneği verilmiştir:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

AG_NEWS veri setimizin vektör temsilinden bag-of-words vektörünü hesaplamak için aşağıdaki fonksiyonu kullanabiliriz:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Not:** Burada varsayılan kelime dağarcığı boyutunu belirtmek için global `vocab_size` değişkenini kullanıyoruz. Kelime dağarcığı boyutu genellikle oldukça büyük olduğundan, kelime dağarcığı boyutunu en sık kullanılan kelimelerle sınırlayabiliriz. `vocab_size` değerini düşürmeyi deneyin ve aşağıdaki kodu çalıştırarak bunun doğruluğu nasıl etkilediğini görün. Daha yüksek performans karşılığında doğrulukta bir düşüş beklemelisiniz, ancak dramatik bir düşüş olmamalı.


## BoW sınıflandırıcısını eğitme

Artık metnimizin Bag-of-Words (BoW) temsilini nasıl oluşturacağımızı öğrendiğimize göre, bunun üzerine bir sınıflandırıcı eğitelim. Öncelikle, veri setimizi eğitime uygun bir şekilde dönüştürmemiz gerekiyor; böylece tüm konumsal vektör temsilleri Bag-of-Words temsiline dönüştürülmüş olacak. Bu, standart torch `DataLoader`'a `collate_fn` parametresi olarak `bowify` fonksiyonunu geçirerek gerçekleştirilebilir:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Şimdi, bir doğrusal katman içeren basit bir sınıflandırıcı sinir ağı tanımlayalım. Girdi vektörünün boyutu `vocab_size`'a eşittir ve çıktı boyutu sınıf sayısına (4) karşılık gelir. Sınıflandırma görevini çözdüğümüz için, son aktivasyon fonksiyonu `LogSoftmax()`dır.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Şimdi standart PyTorch eğitim döngüsünü tanımlayacağız. Veri setimiz oldukça büyük olduğu için, eğitim amacıyla yalnızca bir epoch boyunca eğitim yapacağız ve bazen bir epoch'tan daha az süreyle eğitim yapacağız (`epoch_size` parametresini belirlemek, eğitimi sınırlamamıza olanak tanır). Ayrıca, eğitim sırasında biriken eğitim doğruluğunu raporlayacağız; raporlama sıklığı `report_freq` parametresi kullanılarak belirlenir.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrams, TriGrams ve N-Grams

Bir bag of words yaklaşımının bir sınırlaması, bazı kelimelerin çok kelimeli ifadelerin bir parçası olmasıdır. Örneğin, 'hot dog' kelimesi, diğer bağlamlarda 'hot' ve 'dog' kelimelerinden tamamen farklı bir anlama sahiptir. Eğer 'hot' ve 'dog' kelimelerini her zaman aynı vektörlerle temsil edersek, bu modelimizi yanıltabilir.

Bunu ele almak için, **N-gram temsilleri** genellikle belge sınıflandırma yöntemlerinde kullanılır. Burada, her bir kelimenin, iki kelimelik veya üç kelimelik ifadelerin sıklığı, sınıflandırıcıları eğitmek için faydalı bir özelliktir. Örneğin, bigram temsilde, orijinal kelimelere ek olarak tüm kelime çiftlerini de kelime dağarcığına ekleyeceğiz.

Aşağıda, Scikit Learn kullanarak bir bigram bag of words temsili oluşturmanın bir örneği verilmiştir:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

N-gram yaklaşımının en büyük dezavantajı, kelime dağarcığının çok hızlı bir şekilde büyümeye başlamasıdır. Pratikte, N-gram temsilini *gömüler* gibi bazı boyut indirgeme teknikleriyle birleştirmemiz gerekir; bu konuyu bir sonraki bölümde ele alacağız.

**AG News** veri setimizde N-gram temsilini kullanmak için özel bir ngram kelime dağarcığı oluşturmamız gerekiyor:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Aynı kodu yukarıda olduğu gibi sınıflandırıcıyı eğitmek için kullanabilirdik, ancak bu çok bellek verimsiz olurdu. Bir sonraki bölümde, gömme yöntemlerini kullanarak bigram sınıflandırıcıyı eğiteceğiz.

> **Not:** Metinde belirtilen sayının üzerinde geçen ngram'ları bırakabilirsiniz. Bu, nadir bigramların göz ardı edilmesini sağlayacak ve boyutluluğu önemli ölçüde azaltacaktır. Bunu yapmak için `min_freq` parametresini daha yüksek bir değere ayarlayın ve kelime dağarcığının uzunluğundaki değişimi gözlemleyin.


## Terim Frekansı Ters Doküman Frekansı (TF-IDF)

BoW (Bag of Words) temsili, kelime oluşumlarını kelimenin kendisinden bağımsız olarak eşit şekilde ağırlıklandırır. Ancak, *bir*, *içinde* gibi sık kullanılan kelimelerin, özel terimlere kıyasla sınıflandırma için çok daha az önemli olduğu açıktır. Aslında, çoğu NLP görevinde bazı kelimeler diğerlerinden daha önemlidir.

**TF-IDF**, **terim frekansı–ters doküman frekansı** anlamına gelir. Bu, bag of words yönteminin bir varyasyonudur. Burada, bir kelimenin bir dokümanda bulunup bulunmadığını gösteren ikili 0/1 değeri yerine, kelimenin korpustaki oluşum sıklığıyla ilişkili bir kayan nokta değeri kullanılır.

Daha resmi bir şekilde, bir kelimenin $i$ doküman $j$'deki ağırlığı $w_{ij}$ şu şekilde tanımlanır:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
burada
* $tf_{ij}$, $i$ kelimesinin $j$ dokümanında kaç kez geçtiğini, yani daha önce gördüğümüz BoW değerini ifade eder
* $N$, koleksiyondaki doküman sayısını ifade eder
* $df_i$, $i$ kelimesini içeren dokümanların tüm koleksiyondaki sayısını ifade eder

TF-IDF değeri $w_{ij}$, bir kelimenin bir dokümanda kaç kez geçtiğiyle orantılı olarak artar ve kelimenin korpustaki dokümanlarda bulunma sayısına göre dengelenir. Bu, bazı kelimelerin diğerlerinden daha sık görünmesi gerçeğini ayarlamaya yardımcı olur. Örneğin, bir kelime koleksiyondaki *her* dokümanda geçiyorsa, $df_i=N$ olur ve $w_{ij}=0$ olur; bu terimler tamamen göz ardı edilir.

Scikit Learn kullanarak metnin TF-IDF vektörleştirmesini kolayca oluşturabilirsiniz:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Sonuç

Ancak, TF-IDF temsilleri farklı kelimelere sıklık ağırlığı verse de anlamı veya sıralamayı temsil edemezler. Ünlü dilbilimci J. R. Firth'in 1935'te söylediği gibi, “Bir kelimenin tam anlamı her zaman bağlamsaldır ve bağlamdan bağımsız bir anlam çalışması ciddiye alınamaz.”. Kursun ilerleyen bölümlerinde, dil modelleme kullanarak metinden bağlamsal bilgiyi nasıl yakalayacağımızı öğreneceğiz.



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlık içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul etmiyoruz.
